In [1]:
import re
import pickle
import pandas as pd
import numpy as np
from numpy import nan

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [4]:
#PREPROCESSING
def preprocess(col):
    col_l = []
    for r in col:
        #print(r)
        try:
            a = r.lower() 
            a = a.replace("'s", "")
            #a = a.replace(["\n"], "hjfgjh")
            a = re.sub('[^a-zA-Z0-9 \n\.]', '',a)
            a = re.sub('[:.\n\t]', '',a)
            a = re.sub('[?:!.+,]]][]]//;]', '',a) 
            
            col_l.append(a)
        except:
            #print("No header found")
            col_l.append("No Header")
            pass

    # Downloading punkt and wordnet from NLTK
    nltk.download('punkt')
    print("------------------------------------------------------------")
    nltk.download('wordnet')
    
    wordnet_lemmatizer = WordNetLemmatizer()
    
    nltk.download('stopwords')
    
    lemmatized_text_list = []

    for s in col_l:
        #print(s)
        # Create an empty list containing lemmatized words
        lemmatized_list = []

        # Save the text and its words into an object
        #text = df.loc[row]['Content_Parsed_4']
        text_words = s.split(" ")

        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        # Join the list
        lemmatized_text = " ".join(lemmatized_list)

        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)
    
    new_lemmatized_text_list = []
    for x in lemmatized_text_list:
        new_lemmatized_text_list.append(" ".join(x.split())) 
    
    pl = pd.Series(new_lemmatized_text_list)
   # pl = pd.Series(col_l)
    stop_words = list(stopwords.words('english'))
    
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        #pl = pd.Series(col_l)
   
        pl = pl.str.replace(regex_stopword, '')
        
    
    return pl.tolist()

In [5]:
############Read the new dataset for prediction#########

In [6]:
#dfnew= pd.read_csv('dataset75iotnoniot.csv')##75  iotnoniot projects for prediction  
dfnew= pd.read_csv('dataset75iot.csv')#75  iot projects for prediction 
#dfnew= pd.read_csv('dataset75noniot.csv')#75  noniot projects for prediction 

In [7]:
cnew=(dfnew['descr']).tolist()

In [8]:
connew=preprocess(cnew)
print(connew)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


------------------------------------------------------------
['   lot  work  image process  need  test cameras imagers   go  create  scalable test platform read    project     lot  work  image process  need  test cameras imagers   go  create  scalable test platform   look  create image process solutions previously  use image sensors  use  parallel interface  cameras  use hdmi interfaces one common interface  image sensors  mipi  let take  look     build  image process design   zybo z7   capable  use  hdmi input   mipi camera interface  also want  lay  grind work   able  create  image process system  allow   quickly  easily test cameras  sensors    future upgrade   project    support  parallel pmod camera  well     able  select   imager  wish  use   algorithm    add advantage   pin    zybo mipi connector   align   commonly use maker cameras  example  pi cameras   look     use  pi camera  another project mipi   commonly use abbreviation   mobile industry processor interface    interface 

In [9]:
df1 = pd.DataFrame({"descr":connew})
df1

,descr
0,lot work image process need test camera...
1,project want share today arduino maze g...
2,pantry automatically replenish kitchen item...
3,make lanethernet relay switch use arduino un...
4,cellular backdoor hologram nova raspberry pi...
...,...
70,quick start guide use esp8266 iot devic...
71,monitor door valve gas open someone ope...
72,internet things mozilla wot gateway base r...
73,ball fetcher robot make use tupperware rasp...


In [24]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=(1,2),
                        stop_words=None,
                        lowercase=False,
                        max_df=10,
                        min_df=1,
                        max_features=300,
                        norm='l2',
                        sublinear_tf=True)

In [25]:
pred_test = tfidf.fit_transform(connew).toarray()
print(pred_test)


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.11899114 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.4142179 ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [26]:
################################## DECISION TREE ###############################################

In [27]:
# load the model from disk
loaded_model = pickle.load(open("finalized_model.sav", 'rb'))

DTC_pred1 = loaded_model.predict(pred_test)
DTC_pred1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [28]:
loaded_naive = pickle.load(open("finalized_naive.sav", 'rb'))

Naive_pred1 = loaded_naive.predict(pred_test)
Naive_pred1

array([1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [29]:
loaded_svm = pickle.load(open("finalized_svm.sav", 'rb'))

svm_pred1 = loaded_svm.predict(pred_test)
svm_pred1

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0], dtype=int64)

In [30]:
loaded_knn = pickle.load(open("finalized_knn.sav", 'rb'))
knn_pred1 = loaded_knn.predict(pred_test)
knn_pred1

array([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [31]:
loaded_sgd = pickle.load(open("finalized_sgd.sav", 'rb'))
sgd_pred1 = loaded_sgd.predict(pred_test)
sgd_pred1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0], dtype=int64)

In [32]:
loaded_random = pickle.load(open("finalized_random.sav", 'rb'))
random_pred1 = loaded_random.predict(pred_test)
random_pred1

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0], dtype=int64)